### To do:
- OK mudar estrutura de set para lista ordenada
- OK implementar and/or
- usar dataset imdb
- implementar tf-df
- implementar filtros
- implementar sort

In [1]:
docs = [
    {
        'id': 1,
        'title': 'Star Wars',
    },
    {
        'id': 2,
        'title': 'Star Wars IV',
    },
    {
        'id': 3,
        'title': 'Star Wars V',
    },
    {
        'id': 4,
        'title': 'Star Trek',
    },
    {
        'id': 5,
        'title': 'Star Wars: the Clone Wars',
    },
    {
        'id': 6,
        'title': 'Toy Story',
    },
    {
        'id': 7,
        'title': 'Solo: a Star Wars story',
    },
]

In [2]:
import nltk
import unidecode
import string

nltk.download('stopwords')
nltk.download('punkt')
  
stopwords = set(nltk.corpus.stopwords.words('english'))

def analyse(text):
    text = ''.join([t for t in text if t not in string.punctuation])
    text = text.lower().strip()
    text = unidecode.unidecode(text)
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords]
    return tokens

analyse('Star Wars: the Clone Wars')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leonardo.wajnsztok/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/leonardo.wajnsztok/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['star', 'wars', 'clone', 'wars']

In [38]:
from collections import defaultdict
from sortedcontainers import SortedKeyList
import numpy


class TokenInfo:
    def __init__(self, term, doc_id):
        self.term = term
        self.doc_id = doc_id
        self.tf = 1
        
    def __repr__(self):
        return f'({self.doc_id}, tf: {self.tf})'
    
class Index:
    def __init__(self, documents=None):
        self.index = defaultdict(lambda: SortedKeyList(key=lambda x: x.doc_id))
        self.df = defaultdict(int)
        self.documents = {}
        if documents:
            self.build(documents)

        
    def __repr__(self):
        rep = ['term freq:']
        for k, v in self.index.items():
            rep.append(f'{k}: {v}')
        rep.append('\n')
        rep.append(f'document freq:\n{dict(self.df)}')
        return '\n'.join(rep)
        
    def build(self, documents):
        for doc in documents:
            self.add_document(doc)
            
    def add_document(self, doc):
        text = doc['title']
        doc_id = doc['id']
        self.documents[doc_id] = doc
        
        tokens = analyse(text)
        tokens_set = set()
        
        for t in tokens:
            if t not in tokens_set:
                tokens_set.add(t)
                self.df[t] += 1
                
            term_docs = self.index[t]
            term_info_idx = self._find_doc_in_list(term_docs, doc_id)
            if term_info_idx:
                term_docs[term_info_idx].tf += 1
            else:
                term_docs.add(TokenInfo(t, doc_id))
                
    def _find_doc_in_list(self, elements, value):
        index = elements.bisect_key_left(value)
        if index < len(elements) and elements[index].doc_id == value:
            return index
                
    def _and(self, docs1, docs2):
        intersection = []
        i, j = 0,0
        while i < len(docs1) and j < len(docs2):
            if docs1[i] == docs2[j]:
                intersection.append(docs1[i])
                i += 1
                j += 1
            elif docs1[i] > docs2[j]:
                j += 1
            else:
                i += 1
        return intersection
    
    def _or(self, docs1, docs2):
        union = []
        i, j = 0,0
        while i < len(docs1) and j < len(docs2):
            if docs1[i] == docs2[j] :
                union.append(docs1[i])
                i += 1
                j += 1
            elif docs1[i] > docs2[j]:
                union.append(docs2[j])
                j += 1
            else:
                union.append(docs1[i])
                i += 1
        
        while i < len(docs1):
            union.append(docs1[i])
            i += 1

        while j < len(docs2):
            union.append(docs2[j])
            j += 1
            
        return union
    
    def _get_docs(self, docs):
        return [self.documents[d] for d in docs]
                
    def search(self, text, operation='AND'):
        tokens = analyse(text)
        found_docs = []
        
        for t in tokens:
            if t in self.index:
                new_docs = [d.doc_id for d in self.index[t]]
                
                if found_docs:
                    if operation == 'AND':
                        found_docs = self._and(found_docs, new_docs)
                    elif operation == 'OR':
                        found_docs = self._or(found_docs, new_docs)
                    else:
                        raise NotImplementedError(f'Operation "{operation}" not implemented!')
                else:
                    found_docs.extend(new_docs)

        return self._get_docs(found_docs)
    

index = Index(docs)

In [39]:
index.search('toy wars', operation='AND')

[]

In [42]:
index.search('toy', operation='AND')

[{'id': 6, 'title': 'Toy Story'}]

In [43]:
index.search('toy wars', operation='OR')

[{'id': 1, 'title': 'Star Wars'},
 {'id': 2, 'title': 'Star Wars IV'},
 {'id': 3, 'title': 'Star Wars V'},
 {'id': 5, 'title': 'Star Wars: the Clone Wars'},
 {'id': 6, 'title': 'Toy Story'},
 {'id': 7, 'title': 'Solo: a Star Wars story'}]